In [19]:
table = hdul[1].data

spectroscopic_epochs = table['SPECTRO_DUPLICATE']
spectroscopic_epochs = [list(set(i)) for i in spectroscopic_epochs]

sdss_indices = []

for idx, arr in enumerate(spectroscopic_epochs):
    if 1 in arr: 
        sdss_indices.append(idx)

sdss_indices = np.array(sdss_indices)

sdss_names = table['SDSS_NAME'][sdss_indices]
plates, mjds, fiberIDs = table['PLATE'][sdss_indices], table['MJD'][sdss_indices], table['FIBERID'][sdss_indices]

mask = np.argsort(plates)

sdss_names, plates, mjds, fiberIDs = sdss_names[mask], plates[mask], mjds[mask], fiberIDs[mask]

In [12]:
import os 
from tqdm import tqdm 
import os
import spender
import torch
from accelerate import Accelerator

# hardware optimization
accelerator = Accelerator(mixed_precision='fp16')

# get code, instrument, and pretrained spectrum model from the hub
sdss, model = spender.hub.load('sdss_II',  map_location=accelerator.device)

# prep data download stuff
download_dir = r'C:\Users\tkiker\Documents\GitHub\AGN-UMAP\code\monthly\july2024\wk4\sdss\data'

sort_mask = np.argsort(plates)

all_ids = np.array([plates, mjds, fiberIDs]).T
all_ids = np.array([tuple(i) for i in all_ids])[sort_mask]

In [27]:
import requests

for id in tqdm(all_ids): 
    fiberID = str(id[2])
    if len(fiberID) < 4: 
        fiberID = (4-len(fiberID))*'0' + fiberID
    
    plate = str(id[0])
    if len(plate) < 4: 
        plate = (4-len(plate))*'0' + plate

    mjd = str(id[1])
    
    plate_dir = os.path.join(download_dir, plate)

    # https://data.sdss.org/sas/dr17/eboss/spectro/redux/v5_13_2/spectra/full/3586/spec-3586-55181-0890.fits
    file_url = f'https://data.sdss.org/sas/dr17/eboss/spectro/redux/v5_13_2/spectra/full/{plate}/spec-{plate}-{mjd}-{fiberID}.fits'
    
    try:
        response = requests.get(file_url)
        response.raise_for_status()
        
        os.makedirs(plate_dir, exist_ok=True)
        file_path = os.path.join(plate_dir, f'spec-{plate}-{mjd}-{fiberID}.fits')

        with open(file_path, 'wb') as file:
            file.write(response.content)
        
        #print(f"Downloaded file and saved to {file_path}")

    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except Exception as err:
        print(f"An error occurred: {err}")

 37%|███▋      | 10540/28252 [3:33:09<5:17:54,  1.08s/it]

In [22]:
file_url = f'https://data.sdss.org/sas/dr17/eboss/spectro/redux/v5_13_2/spectra/full/3586/spec-3586-55181-0890.fits'
    
plate = '3586'
mjd = '55181'
fiberID = '0890'

try:
    response = requests.get(file_url)
    response.raise_for_status()
    
    os.makedirs(plate_dir, exist_ok=True)
    file_path = os.path.join(plate_dir, f'spec-{plate}-{mjd}-{fiberID}.fits')

    with open(file_path, 'wb') as file:
        file.write(response.content)
    
    print(f"Downloaded file and saved to {file_path}")
except requests.exceptions.HTTPError as http_err:
    print(f"HTTP error occurred: {http_err}")
except Exception as err:
    print(f"An error occurred: {err}")

Downloaded file and saved to C:\Users\tkiker\Documents\GitHub\AGN-UMAP\code\monthly\july2024\wk4\sdss\data\3587\spec-3586-55181-0890.fits
